# Justin Drake APR

In [1]:
import setup

time: 12.8 ms (started: 2021-05-20 11:27:48 +02:00)


In [2]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

time: 285 ms (started: 2021-05-20 11:27:48 +02:00)


In [3]:
from experiments.run import run
import visualizations as visualizations
from model.parameters import parameters

time: 686 ms (started: 2021-05-20 11:27:48 +02:00)


In [4]:
from experiments.default import experiment, TIMESTEPS, DELTA_TIME

time: 19.4 ms (started: 2021-05-20 11:27:49 +02:00)


In [5]:
from model.types import Phase

time: 20 ms (started: 2021-05-20 11:27:49 +02:00)


In [6]:
daily_transactions = parameters["daily_transactions_process"][0]()
daily_transactions

1400000

time: 26.4 ms (started: 2021-05-20 11:27:49 +02:00)


In [7]:
transaction_average_gas = parameters["transaction_average_gas"][0]
transaction_average_gas

73123

time: 19.2 ms (started: 2021-05-20 11:27:49 +02:00)


In [8]:
# Pre-process the basefee and tip process values from Justin Drake scenarios:
# https://docs.google.com/spreadsheets/d/1FslqTnECKvi7_l4x6lbyRhNtzW9f6CVEzwDf04zprfA

# Scenarios: optimistic, lean optimistic, best guess, lean conservative, conservative
eth_staked_scenarios = [4.5e6, 5e6, 6e6, 10e6, 15e6]
daily_evm_fees_scenarios = [15e3, 12e3, 10e3, 8e3, 6e3]
fee_burn_percentage_scenarios = [0.50, 0.60, 0.70, 0.75, 0.80]

basefee_values = []
tip_values = []

for scenario in range(len(eth_staked_scenarios)):
    basefee_pct = fee_burn_percentage_scenarios[scenario]
    tip_pct = 1 - basefee_pct
    evm_fees = daily_evm_fees_scenarios[scenario]
    
    basefee_plus_tip = (evm_fees * 1e9) / (daily_transactions * transaction_average_gas)
    basefee = basefee_plus_tip * basefee_pct
    tip = basefee_plus_tip * tip_pct
    
    basefee_values.append(basefee)
    tip_values.append(tip)
    
    print(daily_transactions * transaction_average_gas * (basefee + tip) / 1e9)

15000.000000000002
12000.0
10000.0
8000.0
6000.0
time: 18.8 ms (started: 2021-05-20 11:27:49 +02:00)


In [9]:
parameter_overrides = {
    "phase": [Phase.POST_MERGE],
    "eth_price_process": [lambda _run=None, _timestep=None: 3000],
    "eth_staked_process": [lambda _run=None, _timestep=None, eth_staked=eth_staked: eth_staked for eth_staked in eth_staked_scenarios],  # ETH
    "eip1559_basefee_process": [lambda _run=None, _timestep=None, basefee=basefee: basefee for basefee in basefee_values],  # Gwei per gas
    "eip1559_tip_process": [lambda _run=None, _timestep=None, tip=tip: tip for tip in tip_values],  # Gwei per gas
}

# Override default experiment parameters
experiment.simulations[0].model.params.update(parameter_overrides)
# Run single timestep, set unit of time to multiple epochs
experiment.simulations[0].timesteps = 1
experiment.simulations[0].model.params.update({"dt": [TIMESTEPS * DELTA_TIME]})

time: 18.5 ms (started: 2021-05-20 11:27:49 +02:00)


In [10]:
df, exceptions = run(experiment)

2021-05-20 11:27:49,264 - root - INFO - Running experiment
2021-05-20 11:27:49,335 - root - INFO - Starting simulation 0 / run 0 / subset 0
2021-05-20 11:27:49,352 - root - INFO - Starting simulation 0 / run 0 / subset 3
2021-05-20 11:27:49,340 - root - INFO - Starting simulation 0 / run 0 / subset 1
2021-05-20 11:27:49,347 - root - INFO - Starting simulation 0 / run 0 / subset 2
2021-05-20 11:27:49,358 - root - INFO - Starting simulation 0 / run 0 / subset 4
2021-05-20 11:27:49,379 - root - INFO - Experiment complete
time: 206 ms (started: 2021-05-20 11:27:49 +02:00)


In [11]:
df

,phase,timestamp,eth_price,eth_supply,eth_staked,supply_inflation,network_issuance,pow_issuance,number_of_validators_in_activation_queue,average_effective_balance,...,supply_inflation_pct,total_revenue_yields_pct,total_profit_yields_pct,total_online_validator_rewards_eth,total_tips_to_validators_eth,source_reward_eth,target_reward_eth,head_reward_eth,block_proposer_reward_eth,sync_reward_eth
1,4.0,2021-05-20 11:27:48.856600,3000,1.131440e+08,4500000.0,-0.021029,-2.394822e+06,0,230,3.200000e+10,...,-2.102938,67.755511,64.051432,3.005218e+06,2700000.0,58006.66869,116013.337379,58006.66869,39347.620747,38671.11246
3,4.0,2021-05-20 11:27:48.856600,3000,1.132520e+08,5000000.0,-0.020081,-2.286822e+06,0,230,3.200000e+10,...,-2.008101,41.256760,38.988142,2.033218e+06,1728000.0,58006.66869,116013.337379,58006.66869,39347.620747,38671.11246
5,4.0,2021-05-20 11:27:48.856600,3000,1.133240e+08,6000000.0,-0.019449,-2.214822e+06,0,230,3.200000e+10,...,-1.944877,23.423300,22.124481,1.385218e+06,1080000.0,58006.66869,116013.337379,58006.66869,39347.620747,38671.11246
7,4.0,2021-05-20 11:27:48.856600,3000,1.136840e+08,10000000.0,-0.016288,-1.854822e+06,0,230,3.200000e+10,...,-1.628754,10.401535,9.819476,1.025218e+06,720000.0,58006.66869,116013.337379,58006.66869,39347.620747,38671.11246
9,4.0,2021-05-20 11:27:48.856600,3000,1.141160e+08,15000000.0,-0.012494,-1.422822e+06,0,230,3.200000e+10,...,-1.249407,4.986387,4.703537,7.372183e+05,432000.0,58006.66869,116013.337379,58006.66869,39347.620747,38671.11246


time: 44.3 ms (started: 2021-05-20 11:27:49 +02:00)


In [12]:
df['total_revenue_yields'] * 100

1    67.755511
3    41.256760
5    23.423300
7    10.401535
9     4.986387
Name: total_revenue_yields, dtype: float64

time: 20.8 ms (started: 2021-05-20 11:27:49 +02:00)
